In [1]:
%load_ext Cython
import numpy as np
import matplotlib.pyplot as plt

%matplotlib notebook
%load_ext autoreload
%load_ext autotime
%autoreload 2

time: 253 µs


In [2]:
%%cython
import numpy as np
cimport numpy as np

cdef double maxtune = 0.3 # The maximum tune tested

cdef int shift = 0 # Number of turns to calculate before starting to store tracking values
cdef int N = 2000 # Number of turns to calculate for each particle after the shift
cdef int Ntune = 3000 # Number of tune samples to take

pyshift = int(shift) # Python variable to store shift to use outsite cell
pyN = int(N) # Python variable to store number of turns to use outside cell
pyNtune = int(Ntune)
pymaxtune = float(maxtune)

a=0 # Creating alpha values
b=0.0025 # Creating beta values
g=0 # Creating gamma values

aa,bb,gg=np.meshgrid(a,b,g) # Creating grid of test values

aa=aa.flatten() # Reshaping aa
bb=bb.flatten() # Reshaping bb
gg=gg.flatten() # Reshaping gg

aarr = []
barr = []
garr = []

cdef double tune
cdef int i
for tune in np.linspace(0, maxtune, Ntune):
    
    atrack = []
    btrack = []
    gtrack = []
    
    mu = 2*np.pi*tune # mu of the system
    twiss = np.array([aa, bb, gg], dtype=np.float64)
    M = np.array([[np.cos(mu), np.sin(mu)],[-np.sin(mu),np.cos(mu)]]) # x px transformation matrix
    mat = np.array([[M[0,0]*M[0,0],-2*M[0,0]*M[0,1],M[0,1]*M[0,1]],[-M[0,0]*M[1,0],M[0,0]*M[1,1]+M[0,1]*M[1,0],-M[0,1]*M[1,1]],[M[1,0]*M[1,0],-2*M[1,0]*M[1,1],M[1,1]*M[1,1]]], dtype=np.float64)

    for i in range(shift):
        # Particle tracking for "shift" number of turns
        twiss = mat.dot(twiss)

    for i in range(N):
        # particle tracking for N number of turns
        twiss = mat.dot(twiss)
        atrack.append(twiss[0])
        btrack.append(twiss[1])
        gtrack.append(twiss[2])
    
    aarr.append(atrack)
    barr.append(btrack)
    garr.append(gtrack)
    
aarr = np.array(aarr)  # Converting arrays to numpy arrays for slicing to work
barr = np.array(barr)
garr = np.array(garr)

time: 14.3 s


In [3]:
print(np.shape(aarr))
print(np.shape(barr.T[0,:,:]))

(3000, 2000, 1)
(2000, 3000)
time: 1.19 ms


In [4]:
from sklearn.model_selection import train_test_split

testin, trainin, testout, trainout = train_test_split(barr.T[0,:,:], np.linspace(0, pymaxtune, pyNtune), test_size=0.33, random_state=42)

testin = testin.T
trainin = trainin.T
testout = testout.T
trainout = trainout.T

ValueError: Found input variables with inconsistent numbers of samples: [2000, 3000]

time: 311 ms


In [ ]:
 from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(trainin.T, trainout.T)

In [ ]:
pred = model.predict(testin.T)
pred = pred.T

In [ ]:
fig, ax = plt.subplots()
ax.set_xlabel('$x_o$')
ax.set_ylabel('$p_o$')
ax.set_title("Particle Tracking")

ax.scatter(testout[0,:], testout[1,:], c='b', label="Simulated")
ax.scatter(pred[0,:], pred[1,:], c='r', label="ML Model")

fig.legend()
fig.tight_layout()